# Soluzione problema con flusso di costo minimo, con k = 33 si riescono a raccogliere tutti i passeggeri

In [44]:
import math
import mip
import pandas as pd
import datetime
from pandas import ExcelFile

#acquisizione dei dati dal foglio elettronico
tratte = pd.read_excel('file_di_dati.xls',sheet_name='porzioni di viaggio')
tempi_a_vuoto = pd.read_excel('file_di_dati.xls',sheet_name='tempi di percorrenza a vuoto')

# Definiamo una funzione che usa il mapping dei luoghi con un dizionario che restituisce i tempi a vuoto tra tratte

In [45]:
#funzione per ricavare i tempi di percorrenza a vuoto tra due tratte
nomi = {tempi_a_vuoto.iloc[0][i]:i for i in range(1,17)}

def t_trasf(i,j):
    a = nomi[tratte.iloc[j][2]]
    b = nomi[tratte.iloc[i][3]]
    #matrice riempita per metà...
    if(math.isnan(tempi_a_vuoto.iloc[a][b])):
        return datetime.timedelta(minutes=tempi_a_vuoto.iloc[b][a])
    return datetime.timedelta(minutes=tempi_a_vuoto.iloc[a][b])


# Inizializzazione delle variabili

In [46]:
k = 10
NUMERO_TRATTE = 156 #numero tratte
NUMERO_NODI = NUMERO_TRATTE * 2 + 2 #numero nodi del grafo
A = [[0]*NUMERO_NODI for i in range(NUMERO_NODI)] # archi comprensivi di archi tra i e i' e i' e j
c = [[0]*NUMERO_NODI for i in range(NUMERO_NODI)] # costi relativi agli archi i,j
u = [[0]*NUMERO_NODI for i in range(NUMERO_NODI)] # flussi sugli archi ---> vincoli
b = [0 for i in range(NUMERO_NODI)] # nodi
b[NUMERO_NODI-2] = (-1)*k #sorgente
b[NUMERO_NODI-1] = k      #pozzo


# Assegnamo gli archi in base ai vincoli di compatibilità dei tempi

In [47]:
for i in range(1, NUMERO_TRATTE+1):
    for j in range(i+1, NUMERO_TRATTE+1):
        t = t_trasf(i,j)
        if((tratte.iloc[i,5]+t)<=tratte.iloc[j,4]):
            A[i+NUMERO_TRATTE-1][j-1]=1
            u[i+NUMERO_TRATTE-1][j-1]=1 


In [48]:
#archi interni alle tratte 
for i in range(NUMERO_TRATTE):
    for j in range(NUMERO_TRATTE):
        if(i==j):
            A[i][j+NUMERO_TRATTE]=1
            u[i][j+NUMERO_TRATTE]=1
            

In [49]:
#archi dalla radice al nodo partenza
for j in range(NUMERO_TRATTE):
    A[NUMERO_NODI-2][j]=1
    u[NUMERO_NODI-2][j]=1

In [50]:
#archi dall'arrivo al pozzo
for i in range(NUMERO_TRATTE):
    A[i+NUMERO_TRATTE][NUMERO_NODI-1]=1
    u[i+NUMERO_TRATTE][NUMERO_NODI-1]=1

In [51]:
#arco di ottimizzazione dal pozzo alla radice
A[NUMERO_NODI-2][NUMERO_NODI-1]=1
u[NUMERO_NODI-2][NUMERO_NODI-1]=k     


In [52]:
for i in range(NUMERO_TRATTE):
    for j in range(NUMERO_TRATTE):
        if(i==j):
            c[i][j+NUMERO_TRATTE]=(-1)*(tratte.iloc[i+1][1])
# trasponiamo i costi in una matrice


# Definizione modello e validazione

In [53]:
m = mip.Model()
x = [m.add_var(var_type=mip.INTEGER) for g in range(NUMERO_NODI*NUMERO_NODI)]
m.objective = mip.minimize(mip.xsum(c[i][j]*x[(i*NUMERO_NODI)+j] for j in range(NUMERO_NODI) for i in range(NUMERO_NODI) if A[i][j]>0))

# Vincoli di bilancio per ogni nodo

In [54]:
for i in range(NUMERO_NODI):
    
    BS = []
    for k in range(NUMERO_NODI):
        if(A[k][i]>0):
            BS.append(k)
    FS = []
    for k in range(NUMERO_NODI):
        if(A[i][k]>0):
            FS.append(k)
            
    m.add_constr(  mip.xsum(x[(j*NUMERO_NODI)+i] for j in BS)  -  mip.xsum(x[(i*NUMERO_NODI)+j] for j in FS)  ==  b[i])


for i in range(NUMERO_NODI):
    for j in range(NUMERO_NODI): 
        if(A[i][j]>0):
            m.add_constr(x[(i*NUMERO_NODI)+j]>= 0)


for i in range(NUMERO_NODI):
    for j in range(NUMERO_NODI): 
        if(A[i][j]>0):
            m.add_constr(x[(i*NUMERO_NODI)+j]<= u[i][j])


In [55]:

m.optimize()
print(f"soluzione problema è {-1*(m.objective_value)} passeggeri raccolti")



soluzione problema per 313 è 4492.0 passeggeri raccolti
